In [1]:
import numpy as np
import torch
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(formatter={'float_kind':'{:.4f}'.format})

In [2]:
def transform(df, v):
    if len(df[v].shape) >3:
        df_v = df[v].squeeze(0)
    else:
        df_v = df[v]
    return np.concatenate([df_v[0],df_v[1],df_v[2]])

In [3]:
def att_err(pred, sample, t):
    return abs(pred[t==1].mean() - sample[t==1].mean())

In [4]:
def pehe(pred, true, t):
    return np.square(pred[t==1] - true[t==1]).mean()

In [5]:
def small(x):
    return float("{:.2f}".format(x))

In [6]:
directory = '/Users/claudiashi/result/nice_paper/paper_result/'

In [7]:
def get_att(mod, version, collider, method, all_train='all_train'):
    path = directory+'{}/{}/{}/{}/{}/'.format(method,collider,all_train,mod, version)
    err = {'erm': 0, 'irm':0 }
    for model in ['erm','irm']:
        loss = []
        for copy in range(1, 11):

            ite_df = np.load(path + "{}_ite_output_{}.npz".format(model, str(copy)))
            all_t = transform(ite_df, 'T')
            pred_ite = transform(ite_df, 'pred_ite')
            sample_ite = transform(ite_df, 'sample_ite')
            att_e = att_err(pred_ite, sample_ite, all_t)
            
            loss.append(att_e)
        err[model] = [small(np.mean(loss)), small(np.std(loss))]
    print("{}-{}-{}-{}-att mae is: ".format(mod, version, collider, method), err)
    return err

In [8]:
def get_pehe(mod, version, collider,method):
    path = directory+'{}/{}/{}/{}/{}/'.format(method,collider,  'train_test',mod, version)
    err = {'erm': 0, 'irm':0 }
    for model in ['erm','irm']:
        loss= []
        for copy in range(1, 11):
            ite_df = np.load(path + "{}_ite_output_{}.npz".format(model, str(copy)))
            test_t = ite_df['T'][2]
            test_pred_ite = ite_df['pred_ite'][0][2]
            test_sample_ite = ite_df['sample_ite'][0][2]
            pehe_e = pehe(test_pred_ite, test_sample_ite, test_t)
            loss.append(pehe_e)
            err[model] = (small(np.mean(loss)), small(np.std(loss)))
    print("{}-{}-{}-{}-out of sample pehe is: ".format(mod, version, collider, method), err)


            
        

In [9]:
att_res = {}
for mod in ['Mod1', 'Mod2', 'Mod3','Mod4']:
    att_res[mod] = {}
    for v in ['low','med','high']:
        att_res[mod][v]={}
        for c in ['collider','no_collider']:
            att_res[mod][v][c]={}
            for m in ['tarnet']:
                att_res[mod][v][c][m]=get_att(mod, v, c, method=m, all_train='all_train')



Mod1-low-collider-tarnet-att mae is:  {'erm': [0.31, 0.11], 'irm': [0.1, 0.03]}
Mod1-low-no_collider-tarnet-att mae is:  {'erm': [0.08, 0.06], 'irm': [0.05, 0.05]}
Mod1-med-collider-tarnet-att mae is:  {'erm': [0.37, 0.12], 'irm': [0.09, 0.03]}
Mod1-med-no_collider-tarnet-att mae is:  {'erm': [0.04, 0.02], 'irm': [0.02, 0.02]}
Mod1-high-collider-tarnet-att mae is:  {'erm': [0.28, 0.09], 'irm': [0.09, 0.06]}
Mod1-high-no_collider-tarnet-att mae is:  {'erm': [0.06, 0.05], 'irm': [0.01, 0.01]}
Mod2-low-collider-tarnet-att mae is:  {'erm': [0.36, 0.1], 'irm': [0.1, 0.07]}
Mod2-low-no_collider-tarnet-att mae is:  {'erm': [0.07, 0.1], 'irm': [0.02, 0.02]}
Mod2-med-collider-tarnet-att mae is:  {'erm': [0.33, 0.15], 'irm': [0.13, 0.03]}
Mod2-med-no_collider-tarnet-att mae is:  {'erm': [0.16, 0.17], 'irm': [0.03, 0.03]}
Mod2-high-collider-tarnet-att mae is:  {'erm': [0.32, 0.17], 'irm': [0.11, 0.04]}
Mod2-high-no_collider-tarnet-att mae is:  {'erm': [0.1, 0.12], 'irm': [0.03, 0.03]}
Mod3-low-co

In [10]:
pehe_res = {}
for mod in ['Mod1', 'Mod2', 'Mod3','Mod4']:
    pehe_res[mod] = {}
    for v in ['low','high','med']:
        pehe_res[mod][v]={}
        for c in ['collider', 'no_collider']:
            pehe_res[mod][v][c]={}
            for m in ['dragon','tarnet']:
                pehe_res[mod][v][c][m]=get_pehe(mod, v, c,m)
df1 = pd.DataFrame(pehe_res)


Mod1-low-collider-dragon-out of sample pehe is:  {'erm': (0.25, 0.06), 'irm': (0.09, 0.01)}
Mod1-low-collider-tarnet-out of sample pehe is:  {'erm': (0.18, 0.05), 'irm': (0.08, 0.02)}
Mod1-low-no_collider-dragon-out of sample pehe is:  {'erm': (0.21, 0.07), 'irm': (0.06, 0.02)}
Mod1-low-no_collider-tarnet-out of sample pehe is:  {'erm': (0.15, 0.03), 'irm': (0.07, 0.02)}
Mod1-high-collider-dragon-out of sample pehe is:  {'erm': (0.34, 0.14), 'irm': (0.09, 0.03)}
Mod1-high-collider-tarnet-out of sample pehe is:  {'erm': (0.24, 0.06), 'irm': (0.06, 0.02)}
Mod1-high-no_collider-dragon-out of sample pehe is:  {'erm': (0.14, 0.04), 'irm': (0.06, 0.01)}
Mod1-high-no_collider-tarnet-out of sample pehe is:  {'erm': (0.14, 0.06), 'irm': (0.05, 0.01)}
Mod1-med-collider-dragon-out of sample pehe is:  {'erm': (0.4, 0.08), 'irm': (0.07, 0.01)}
Mod1-med-collider-tarnet-out of sample pehe is:  {'erm': (0.41, 0.08), 'irm': (0.09, 0.02)}
Mod1-med-no_collider-dragon-out of sample pehe is:  {'erm': (0.15